# Problem Statement

In this competition, we use cartographic variables to classify forest categories.

The word 'cartographic' is an adjective which means 'relating to the science or practice of drawing maps'. Example usage: "he started his own cartographic printing company".

We are going to cover the following steps:
1. Import Libraries
2. Model: XGBoost
3. Evaluation: XGBoost
4. Submission: XGBoost
5. Model: XGBoost Linear using gblinear
6. Evaluation: XGBoost Linear
7. Submission: XGBoost Linear

Let's get started.

# Import Libraries

In [ ]:
import pandas as pd
from pathlib import Path
import warnings  
warnings.filterwarnings('ignore')

data_dir = Path('../input/tabular-playground-series-dec-2021/')

df_train = pd.read_csv(
    data_dir / "train.csv",
    index_col='Id'
)

FEATURES = df_train.columns[:-1]
TARGET = df_train.columns[-1]

df_train.head()

The target attribute 'Cover_Type' contains 7 types of Forest Cover (1, 2, 3, 4, 5, 6, 7).

# Model: XGBoost

Let's try out a simple XGBoost model. This algorithm can handle missing values. We use XGBClassifier (instead of XGBRegressor, for instance), since this is a classification problem.

In [ ]:
from xgboost import XGBClassifier

X = df_train.loc[:, FEATURES]
y = df_train.loc[:, TARGET]

model = XGBClassifier(
    max_depth=3,
    subsample=0.5,
    colsample_bytree=0.5,
    n_jobs=-1,
    # Uncomment if you want to use GPU. Recommended for whole training set.
    tree_method='gpu_hist',
    random_state=0,
)

# Evaluation: XGBoost

The evaluation metric is multi-class classification accuracy.

In [ ]:
from sklearn.model_selection import cross_validate
import warnings 
warnings.filterwarnings('ignore')

def score(X, y, model, cv):
    scoring = ["accuracy"]
    scores = cross_validate(
        model, X, y, scoring=scoring, cv=cv, return_train_score=True
    )
    scores = pd.DataFrame(scores).T
    return scores.assign(
        mean = lambda x: x.mean(axis=1),
        std = lambda x: x.std(axis=1),
    )

scores = score(X, y, model, cv=2)

display(scores)

# Submission: XGBoost

In [ ]:
# Fit on full training set
model.fit(X, y)

X_test = pd.read_csv(data_dir / "test.csv", index_col='Id')

# Make predictions
y_pred = pd.Series(
    model.predict(X_test),
    index=X_test.index,
    name=TARGET,
)

# Create submission file
y_pred.to_csv("submission_xgboost.csv")

# Model: XGBoost Linear using gblinear

In [ ]:
from xgboost import XGBClassifier

X = df_train.loc[:, FEATURES]
y = df_train.loc[:, TARGET]

model_gblinear = XGBClassifier(
    n_jobs=-1,
    random_state=0,
    objective ='reg:squarederror', # WARNING -> reg:linear is now deprecated in favor of reg:squarederror
    booster='gblinear'
)

# Evaluation: XGBoost Linear

The evaluation metric is multi-class classification accuracy.

In [ ]:
scores = score(X, y, model_gblinear, cv=2)

display(scores)

# Submission: XGBoost Linear

In [ ]:
# Fit on full training set
model_gblinear.fit(X, y)

X_test = pd.read_csv(data_dir / "test.csv", index_col='Id')

# Make predictions
y_pred = pd.Series(
    model_gblinear.predict(X_test),
    index=X_test.index,
    name=TARGET,
)

# Create submission file
y_pred.to_csv("submission_gblinear.csv")